# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=admin password=admin")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]
print(filepath)

/home/ubuntu/github-repos/data-engineering-projects/data-modeling-with-postgres/data/song_data/A/B/C/TRABCTK128F934B224.json


In [6]:
df = pd.read_json(filepath, typ='series')
df.head(10)

num_songs                            1
artist_id           AR558FS1187FB45658
artist_latitude                   None
artist_longitude                  None
artist_location                       
artist_name                    40 Grit
song_id             SOGDBUF12A8C140FAA
title                            Intro
duration                      75.67628
year                              2003
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data = list(song_data)
print(song_data)

['SOGDBUF12A8C140FAA', 'Intro', 'AR558FS1187FB45658', 2003, 75.67628]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data = list(artist_data)
print(artist_data)

['AR558FS1187FB45658', '40 Grit', '', None, None]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]
print(filepath)

/home/ubuntu/github-repos/data-engineering-projects/data-modeling-with-postgres/data/log_data/2018/11/2018-11-09-events.json


In [13]:
df = pd.read_json(filepath, lines=True)
df.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df_next_song = df[df['page'] == 'NextSong']
df_next_song['timestamp'] = pd.to_datetime(df_next_song['ts'], unit='ms', origin='unix')
df_next_song.head()

/tmp/ipykernel_5094/4068234810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_song['timestamp'] = pd.to_datetime(df_next_song['ts'], unit='ms', origin='unix')


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:06:17.796
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:09:46.796
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:12:27.796
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:14:52.796
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:17:44.796


In [15]:
t = pd.to_datetime(df_next_song['ts'], unit='ms', origin='unix')
t.head()

0   2018-11-09 00:06:17.796
1   2018-11-09 00:09:46.796
2   2018-11-09 00:12:27.796
3   2018-11-09 00:14:52.796
4   2018-11-09 00:17:44.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_df = pd.DataFrame()
time_df['timestamp'] = t
time_df['hour'] = t.dt.hour
time_df['day'] = t.dt.day
time_df['week'] = t.dt.isocalendar().week
time_df['month'] = t.dt.month
time_df['year'] = t.dt.year
time_df['weekday'] = t.dt.weekday

In [17]:
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-09 00:06:17.796,0,9,45,11,2018,4
1,2018-11-09 00:09:46.796,0,9,45,11,2018,4
2,2018-11-09 00:12:27.796,0,9,45,11,2018,4
3,2018-11-09 00:14:52.796,0,9,45,11,2018,4
4,2018-11-09 00:17:44.796,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()
user_df = user_df[user_df['userId'] != '']

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df_next_song.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.timestamp, row.userId, row.level, songid, artistid, \
                     row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.